In [ ]:
#!/usr/bin/env python3
"""
pnl_only.py
───────────
• Replays the Causal-PIP trader (same logic as before).
• Computes *cumulative* PnL (trade - commission) for each instrument
  at every timestep.
• Saves the result to a CSV file (pnl_cum.csv) with a clear tabular
  layout and returns the numpy ndarray for interactive use.

Usage
-----
python pnl_only.py            # writes pnl_cum.csv in the working dir
"""

from pathlib import Path
from typing  import List, Optional, Tuple

import numpy as np
import pandas as pd


# ═════════════ strategy constants ═════════════════════════════════════ #
BEST_N: List[int] = [
    200, 200, 400, 200, 400, 400, 100, 300, 100, 200,
    100, 500, 100, 300, 200, 100, 200, 100, 200, 300,
    100, 200, 500, 100, 400, 100, 100, 400, 100, 100,
    400, 500, 300, 200, 400, 500, 100, 400, 100, 100,
    100, 300, 400, 500, 100, 400, 500, 100, 500, 300,
]
BEST_W: List[int] = [
     10,  10, 100,  10, 100,  10, 100,  25,  10, 100,
    100,  50,  25,  10,  50,  25,  10,  25,  50, 100,
    100,  10,  10,  50,  25,  25,  10,  50,  25, 100,
     25,  10, 100,  50,  25,  10,  50, 100,  50,  25,
     25,  10, 100, 100,  50,  50,  10,  25,  25,  25,
]

POSITION_LIMIT   = 10_000
COMMISSION_RATE  = 0.0005
THR_MIN, THR_MAX, THR_STEP = 0.00, 0.700, 0.0005
THR_GRID         = np.arange(THR_MIN, THR_MAX + THR_STEP, THR_STEP, dtype=float)


# ═════════════ causal-PIP helpers ══════════════════════════════════════ #
def causal_update(px: float,
                  last_extreme: float,
                  direction: Optional[int],
                  thr: float) -> Tuple[int, float]:
    if direction == 0 or direction is None:
        mv = (px - last_extreme) / last_extreme
        if abs(mv) >= thr:
            return (1 if mv > 0 else -1), px
        return 0, last_extreme
    if direction == 1:  # trending up
        if px > last_extreme:
            return 1, px
        if (last_extreme - px) / last_extreme >= thr:
            return -1, px
        return 1, last_extreme
    # direction == -1
    if px < last_extreme:
        return -1, px
    if (px - last_extreme) / last_extreme >= thr:
        return 1, px
    return -1, last_extreme


def best_threshold(window: np.ndarray) -> float:
    best_thr, best_pnl = THR_GRID[0], -np.inf
    for thr in THR_GRID:
        dir_vec = np.zeros(len(window), dtype=np.int8)
        le, d = window[0], 0
        for k in range(1, len(window)):
            d, le = causal_update(window[k], le, d, thr)
            dir_vec[k] = d
        pnl = np.sum(dir_vec[:-1].astype(np.int32) *
                     POSITION_LIMIT *
                     np.diff(window))
        if pnl > best_pnl:
            best_thr, best_pnl = thr, pnl
    return best_thr


# ═════════════ trader implementation ══════════════════════════════════ #
class CausalPIPTrader:
    class _InstrState:
        __slots__ = ("n", "w", "next_calib", "thr", "dir", "last_extreme")
        def __init__(self, n, w):
            self.n, self.w = n, w
            self.next_calib = n
            self.thr: Optional[float] = None
            self.dir = 0
            self.last_extreme: Optional[float] = None

    def __init__(self):
        self._states = [self._InstrState(n, w)
                        for n, w in zip(BEST_N, BEST_W)]
        print("CausalPIPTrader initialised.")

    def Alg(self, prcSoFar: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        nInst, t_seen = prcSoFar.shape
        pos   = np.zeros(nInst, dtype=np.int32)
        conf  = np.zeros(nInst, dtype=float)

        for i in range(nInst):
            st, px = self._states[i], prcSoFar[i, -1]

            if st.last_extreme is None:
                st.last_extreme = px

            if t_seen >= st.next_calib:
                window   = prcSoFar[i, t_seen - st.n : t_seen]
                st.thr   = best_threshold(window)
                st.next_calib += st.w
                print(f"[t={t_seen:>4}] Inst {i:02d}  thr={st.thr:.4f}")

            if st.thr is None:
                continue

            prev_ext          = st.last_extreme
            new_dir, new_ext  = causal_update(px, prev_ext, st.dir, st.thr)
            move_ratio        = (abs(px - prev_ext) / prev_ext) / st.thr
            conf[i]           = min(move_ratio, 1.0)

            st.dir, st.last_extreme = new_dir, new_ext
            pos[i] = new_dir * POSITION_LIMIT

        return pos, conf


# ═════════════ main routine (PnL only) ═════════════════════════════════ #
def compute_cum_pnl(prices: np.ndarray) -> np.ndarray:
    """
    Run the trader and return cumulative PnL matrix (T × nInst).
    Also saves the table as 'pnl_cum.csv'.
    """
    T, nInst = prices.shape
    trader   = CausalPIPTrader()

    pos = np.zeros((T, nInst), dtype=np.int32)
    for t in range(1, T + 1):
        pos[t - 1], _ = trader.Alg(prices[:t, :].T)

    # step PnL
    trade_pnl = np.zeros_like(prices, dtype=float)
    comm_pnl  = np.zeros_like(prices, dtype=float)

    for t in range(1, T):
        diff                 = prices[t] - prices[t - 1]
        trade_pnl[t]         = pos[t - 1] * diff
        comm_pnl[t]          = -COMMISSION_RATE * np.abs(pos[t] - pos[t - 1]) * prices[t]

    cum_pnl = np.cumsum(trade_pnl + comm_pnl, axis=0)

    # tidy DataFrame for downstream work
    df = pd.DataFrame(
        cum_pnl,
        columns=[f"inst_{i:02d}" for i in range(nInst)],
        index=pd.RangeIndex(start=0, stop=T, name="timestep"),
    )
    df.to_csv("pnl_cum.csv")
    print("Saved cumulative PnL to pnl_cum.csv")

    return cum_pnl


# ═════════════ entry point ════════════════════════════════════════════ #
if __name__ == "__main__":
    prices = np.loadtxt(Path("prices.txt"))  # shape (T, nInst)
    compute_cum_pnl(prices)


In [ ]:
#!/usr/bin/env python3
"""
pnl_pip_plot.py
───────────────
Detects and plots causal-PIP turning-points on an instrument’s cumulative
PnL curve.

• Reads a CSV produced by `pnl_only.py` (time rows × instrument columns).
• Uses *exactly* the same causal-PIP logic as the trading model.
• Prints a table of detected PIPs and shows a plot.

CLI usage (works in terminals **and** Jupyter):
    python pnl_pip_plot.py                  # default inst 0, thr 0.10
    python pnl_pip_plot.py  17  0.07        # inst 17, 7 % threshold
"""

from __future__ import annotations
from pathlib import Path
from typing   import List, Optional, Tuple
import argparse
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# ═════════════ default CONFIG (overridable) ════════════════════════════ #
CSV_PATH   = Path("pnl_cum.csv")
DEFAULT_INST = 2
DEFAULT_THR  = 0.1


# ═════════════ argument parsing (Jupyter-safe) ═════════════════════════ #
def get_cli_args() -> Tuple[int, float]:
    """
    Parse up to two positional arguments **and ignore** any unknown --flags
    (e.g. those injected by `ipykernel` inside Jupyter).
    """
    parser = argparse.ArgumentParser(
        prog="pnl_pip_plot.py",
        description="Plot PIP turning-points on a cumulative-PnL series",
        add_help=False,          # keep it minimal; unknown flags are ignored
    )
    parser.add_argument("instrument", nargs="?", type=int,   default=DEFAULT_INST)
    parser.add_argument("threshold",  nargs="?", type=float, default=DEFAULT_THR)
    parsed, _ = parser.parse_known_args()   # *_ ignores stray kernel args*
    return parsed.instrument, parsed.threshold


# ═════════════ causal-PIP helpers (unchanged logic) ════════════════════ #
def causal_update(px: float,
                  last_ext: float,
                  direction: Optional[int],
                  thr: float) -> Tuple[int, float]:

    move = safe_rel_move(px, last_ext)

    if direction == 0 or direction is None:
        if abs(move) >= thr:
            return (1 if move > 0 else -1), px
        return 0, last_ext

    if direction == 1:                       # trending up
        if px > last_ext:                    # new high → keep trend
            return 1, px
        if safe_rel_move(px, last_ext) <= -thr:
            return -1, px                    # down-move ≥ thr → flip
        return 1, last_ext

    # direction == -1  (trending down)
    if px < last_ext:
        return -1, px
    if safe_rel_move(px, last_ext) >= thr:
        return 1, px
    return -1, last_ext



def detect_pips(series: np.ndarray, thr: float) -> Tuple[np.ndarray, np.ndarray]:
    if series.size == 0:
        return np.empty(0, int), np.empty(0, float)

    start = next((k for k, v in enumerate(series) if v != 0.0), 0)
    last_ext, last_idx, direction = series[start], start, 0
    idx, val = [start], [series[start]]

    for k in range(start + 1, len(series)):
        px = series[k]
        new_dir, new_ext = causal_update(px, last_ext, direction, thr)

        if new_dir != direction:
            idx.append(last_idx)
            val.append(last_ext)

        if new_ext != last_ext:
            last_ext, last_idx = new_ext, k
        direction = new_dir

    if last_idx not in idx:
        idx.append(last_idx)
        val.append(last_ext)

    return np.asarray(idx, int), np.asarray(val, float)


# ═════════════ plotting utility ════════════════════════════════════════ #
def plot_pnl_with_pips(series: np.ndarray,
                       pip_idx: np.ndarray,
                       pip_val: np.ndarray,
                       instr_id: int,
                       thr: float) -> None:
    t = np.arange(len(series))
    plt.figure(figsize=(10, 4))
    plt.plot(t, series, lw=1.2, label="Cumulative PnL")
    plt.axhline(0, color="black", lw=0.6)
    plt.scatter(pip_idx, pip_val,
                s=32, marker="o", facecolors="none",
                edgecolors="blue", lw=1.4,
                label=f"PIPs (thr={thr:.0%})")
    plt.title(f"Instrument {instr_id:02d} — Cumulative PnL with PIPs")
    plt.xlabel("Timestep")
    plt.ylabel("PnL")
    plt.legend(frameon=False)
    plt.tight_layout()
    plt.show()


# ═════════════ main routine ════════════════════════════════════════════ #
def main() -> None:
    inst, thr = get_cli_args()

    # 1) load PnL table
    df = pd.read_csv(CSV_PATH, index_col=0)
    if inst >= df.shape[1]:
        raise IndexError(f"Instrument index {inst} out of range (0 … {df.shape[1]-1})")

    series = df.iloc[:, inst].to_numpy()

    # 2) detect PIPs
    pip_idx, pip_val = detect_pips(series, thr)

    # 3) print results
    print(f"\nDetected PIPs — Instrument {inst:02d}  (threshold {thr:.0%})")
    print(pd.DataFrame({"timestep": pip_idx, "pnl_value": pip_val})
          .to_string(index=False))

    # 4) plot
    plot_pnl_with_pips(series, pip_idx, pip_val, inst, thr)


# ═════════════ entry point ═════════════════════════════════════════════ #
if __name__ == "__main__":
    main()
